In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from torchvision import datasets, models, transforms
import os

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_dir = '/content/drive/MyDrive/SRGAN/result'
TEST = 'test_HR'

# VGG-16 Takes 224x224 images as input, so we resize all of them
data_transforms = {
    TEST: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel grayscale
        transforms.ToTensor(),  # Convert PIL image to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize tensor
    ])
}

image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x),
        transform=data_transforms[x]
    )
    for x in [TEST]
}

dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=8,
        shuffle=True, num_workers=4
    )
    for x in [TEST]
}

dataset_sizes = {x: len(image_datasets[x]) for x in [TEST]}

for x in [TEST]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))

print("Classes: ")
class_names = image_datasets[TEST].classes
print(image_datasets[TEST].classes)

Loaded 3546 images under test_HR
Classes: 
['DME', 'DRUSEN']


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [9]:
import torch
import torchvision.models as models
import torch.nn as nn

# Load the pretrained model
model_A = models.vgg16_bn(pretrained=False)  # Set pretrained=False since you are loading custom weights

# Modify the classifier for 2 classes
num_features = model_A.classifier[6].in_features
features = list(model_A.classifier.children())[:-1]  # Remove the last layer
features.append(nn.Linear(num_features, 2))  # Change to your number of classes
model_A.classifier = nn.Sequential(*features)  # Replace the classifier

# Load the model weights
try:
    model_A.load_state_dict(torch.load('/content/drive/MyDrive/SRGAN/Model_A/VGG16_v2-OCT_Retina_half_dataset.pt'))
except Exception as e:
    print(f"Error loading model: {e}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-9-09badc115ad5>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

In [10]:
import torch
import torchvision.models as models
import torch.nn as nn

# Load the pretrained model
model_B = models.vgg16_bn(pretrained=False)  # Set pretrained=False since you are loading custom weights

# Modify the classifier for 2 classes
num_features = model_B.classifier[6].in_features
features = list(model_B.classifier.children())[:-1]  # Remove the last layer
features.append(nn.Linear(num_features, 2))  # Change to your number of classes
model_B.classifier = nn.Sequential(*features)  # Replace the classifier

# Load the model weights
try:
    model_B.load_state_dict(torch.load('/content/drive/MyDrive/SRGAN/Model_B/VGG16_v2-OCT_Retina_half_dataset.pt'))
except Exception as e:
    print(f"Error loading model: {e}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-10-a289e2f978c3>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the function

In [16]:
import numpy as np

def evaluate_model(model, data_loader, device):
    model.to(device)  # Ensure the model is on the right device
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move inputs and labels to GPU

            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)  # Get probabilities
            preds = torch.argmax(probs, dim=1)     # Get class predictions

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())  # Append probabilities as a list

    return all_labels, all_preds, np.vstack(all_probs)  # Convert list of arrays to a single 2D array


In [17]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# Evaluate both models on the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
true_labels, preds_model1, probs_model1 = evaluate_model(model_A, dataloaders[TEST], device)
true_labels, preds_model2, probs_model2 = evaluate_model(model_B, dataloaders[TEST], device)

# Calculate Accuracy
accuracy_model1 = accuracy_score(true_labels, preds_model1)
accuracy_model2 = accuracy_score(true_labels, preds_model2)

# Calculate F1 Score
f1_model1 = f1_score(true_labels, preds_model1, average='weighted')
f1_model2 = f1_score(true_labels, preds_model2, average='weighted')

# Calculate AUC
# Assuming binary classification; adjust for multi-class as needed
auc_model1 = roc_auc_score(true_labels, probs_model1[:, 1])  # Probability for positive class
auc_model2 = roc_auc_score(true_labels, probs_model2[:, 1])  # Probability for positive class

# Print Results
print("Model 1: Accuracy = {:.4f}, F1 Score = {:.4f}, AUC = {:.4f}".format(accuracy_model1, f1_model1, auc_model1))
print("Model 2: Accuracy = {:.4f}, F1 Score = {:.4f}, AUC = {:.4f}".format(accuracy_model2, f1_model2, auc_model2))


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Model 1: Accuracy = 0.2578, F1 Score = 0.3232, AUC = 0.5233
Model 2: Accuracy = 0.3201, F1 Score = 0.4083, AUC = 0.5115
